In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ast import literal_eval

<b>DATA IMPORTS</b>

In [2]:
current_empl_17_threedigi = pd.read_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/College scorecard/CollegeScorecard_Raw_Data/agged_empl_wages_by_county_THREE DIGIT_2.csv')

In [3]:
six_dig_naics = pd.read_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/CIP NAICS Crosswalk_THREE DIGIT_2.csv')

In [4]:
naicscodes = [int(x) for x in six_dig_naics[['NAICS Code', 'NAICS Title']].drop_duplicates().dropna()['NAICS Code']]
naicstitles = list(six_dig_naics[['NAICS Code', 'NAICS Title']].drop_duplicates().dropna()['NAICS Title'])
naics_codes_to_titles = dict(zip(naicscodes, naicstitles))

In [212]:
unitid_degs_sixdig = pd.read_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/IPEDS/six_deg_ccs_wpercs_no990 051419.csv')

In [155]:
schools = pd.read_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/College scorecard/CollegeScorecard_Raw_Data/merged_all_scorecard_indicators_longitudinal_zips_fipsadj_RESET FROM HERE 051419_adjacents.csv')

In [142]:
schools = schools.replace('NaN', np.nan)

In [7]:
with open('C:/Users/jbachlombardo/Documents/Community Colleges/Data/naics_degree_dictionary_THREEDIGIT_2.pickle', 'rb') as handle:
    naics_deg_dict_THREEDIGIT = pickle.load(handle)

<b>DEF FUNCTIONS</b>

In [187]:
def one_county(s, df_schools, df_employ, df_sixdig_degs, deg_dict) :
    #PREP
    school = df_schools.loc[df_schools['UNITID'] == s]
    fips = school['FIPS'].item()
    county = df_employ.loc[df_employ['FIPS'] == fips]
    school_degs = school.merge(df_sixdig_degs, on = 'UNITID')
    county['annual_avg_emplvl_%'] = county['annual_avg_emplvl'] / county['annual_avg_emplvl'].sum()
    nonzero_jobs = list(county[county['annual_avg_emplvl'] > 0]['industry_code'].unique())
    #CALCS
    school_cols = list(school_degs.columns)
    jobs_for_df = list()
    employment_for_df = list()
    degs_in_job_for_df = list()
    for j in nonzero_jobs :
        deg_list_cc = list()
        for d in deg_dict[j] :
            if str(int(d)) in school_cols :
                deg_list_cc.append(str(int(d)) + '_%')
        degs_produced = school_degs[deg_list_cc].sum(axis = 1).item()
        makeup_deg_list = [deg[:-2] for deg in school_degs[deg_list_cc].dropna(axis = 1).columns]
        jobs_for_df.append(j)
        employment_for_df.append(degs_produced)
        degs_in_job_for_df.append(makeup_deg_list)
    #CLEANUP
    job_comps = pd.DataFrame(data = {'Job': jobs_for_df, '% filled': employment_for_df, 'Degrees makeup': degs_in_job_for_df})
    job_comps = job_comps.set_index('Job').merge(county[['industry_code', 'annual_avg_emplvl_%']].set_index('industry_code'), left_index = True, right_index = True)
    job_comps['% surplus'] = job_comps['% filled'] - job_comps['annual_avg_emplvl_%']
    job_comps['UNITID'] = s
    job_comps = job_comps.reset_index().set_index('UNITID').rename(columns = {'index': 'NAICS Job Category'})
    
    return job_comps

In [188]:
def multiple_counties(s, df_schools, df_employ, df_sixdig_degs, deg_dict) :
    #PREP
    school = df_schools.loc[df_schools['UNITID'] == s]
    counties_fips = literal_eval(schools.loc[schools['UNITID'] == s, 'ADJ_FIPS'].item())
    school_degs = school.merge(df_sixdig_degs, on = 'UNITID')
    counties = df_employ.loc[df_employ['FIPS'].isin(counties_fips)]
    grouped_counties = counties.groupby('industry_code', as_index = False)['annual_avg_emplvl'].sum()
    grouped_counties['annual_avg_emplvl_%'] = grouped_counties['annual_avg_emplvl'] / grouped_counties['annual_avg_emplvl'].sum()
    nonzero_jobs = list(grouped_counties[grouped_counties['annual_avg_emplvl'] > 0]['industry_code'].unique())
    #CALCS
    school_cols = list(school_degs.columns)
    jobs_for_df = list()
    employment_for_df = list()
    degs_in_job_for_df = list()
    for j in nonzero_jobs :
        deg_list_cc = list()
        for d in deg_dict[j] :
            if str(int(d)) in school_cols :
                deg_list_cc.append(str(int(d)) + '_%')
        degs_produced = school_degs[deg_list_cc].sum(axis = 1).item()
        makeup_deg_list = [deg[:-2] for deg in school_degs[deg_list_cc].dropna(axis = 1).columns]
        jobs_for_df.append(j)
        employment_for_df.append(degs_produced)
        degs_in_job_for_df.append(makeup_deg_list)
    #CLEANUP
    job_comps = pd.DataFrame(data = {'Job': jobs_for_df, '% filled': employment_for_df, 'Degrees makeup': degs_in_job_for_df})
    job_comps = job_comps.set_index('Job').merge(grouped_counties[['industry_code', 'annual_avg_emplvl_%']].set_index('industry_code'), left_index = True, right_index = True)
    job_comps['% surplus'] = job_comps['% filled'] - job_comps['annual_avg_emplvl_%']
    job_comps['UNITID'] = s
    job_comps = job_comps.reset_index().set_index('UNITID').rename(columns = {'index': 'NAICS Job Category'})
  
    return job_comps

In [189]:
def get_leftover_degs(s, df_job_comps, df_sixdig_degs) :
    used = list()
    for degs in df_job_comps['Degrees makeup'] :
        if len(degs) > 0 :
            for d in degs :
                if d in used :
                    continue
                else :
                    used.append(d)
    leftover = [c for c in df_sixdig_degs.set_index('UNITID').loc[s].dropna().index if '_%' in c and c[:-2] not in used]
    leftover_df = df_sixdig_degs.set_index('UNITID').loc[s][leftover].to_frame().T.reset_index().rename(columns = {'index': 'UNITID'})
    
    return leftover_df

In [185]:
def plot_job_comps(job_comps, deg_dict) :
    job_comps.set_index('NAICS Job Category').sort_values(by = '% surplus')['% surplus'].plot(kind = 'barh', figsize = (4, 12))

<b>SLIMMING TO NCES DATASET</b>

In [49]:
unique_schools = list(unitid_degs_sixdig['UNITID'])

In [151]:
schools = schools[schools['UNITID'].isin(unique_schools)]

<b>RUNNING COMPS</b>

In [213]:
job_comps_all = pd.DataFrame()
leftovers_all = pd.DataFrame()

In [214]:
for s in unique_schools :
    if pd.isnull(schools.loc[schools['UNITID'] == s, 'ADJ_COUNTIES'].item()) :
        job_comps = one_county(s, schools, current_empl_17_threedigi, unitid_degs_sixdig, naics_deg_dict_THREEDIGIT)
    else :
        job_comps = multiple_counties(s, schools, current_empl_17_threedigi, unitid_degs_sixdig, naics_deg_dict_THREEDIGIT)
    leftovers = get_leftover_degs(s, job_comps, unitid_degs_sixdig)
    job_comps_all = pd.concat([job_comps_all, job_comps])
    leftovers_all = pd.concat([leftovers_all, leftovers])

C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
C:\Users\jbachlombardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future b

In [218]:
job_comps_all.to_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/job_comps 051419_990s removed.csv')
leftovers_all.to_csv('C:/Users/jbachlombardo/Documents/Community Colleges/Data/leftovers 051419_990s removed.csv')

In [216]:
leftovers_all = leftovers_all.set_index('UNITID')
leftovers_all['Total leftover'] = leftovers_all.sum(axis = 1)